In [ ]:
import os
import urllib.request
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
import numpy as np
import tensorflow as tf
from keras.layers import TFSMLayer

app = Flask(__name__)
    
@app.route('/')
def upload_form():
    return render_template(
        'index.html')

@app.route('/', methods=['POST'])
def upload_image():
    file = request.files['file']
    filepath = os.path.join("static/uploads", secure_filename(file.filename))
    file.save(filepath)

    # Preprocess image
    imvar = tf.keras.preprocessing.image.load_img(filepath, target_size=(176, 176))
    imarr = tf.keras.preprocessing.image.img_to_array(imvar)
    imarr = np.expand_dims(imarr, axis=0)  # (1,176,176,3)

    # Load TF SavedModel
    model2 = TFSMLayer("model", call_endpoint="serving_default")

    # Call the model (returns a dict)
    outputs = model2(imarr)
    print("Model outputs:", outputs.keys())   # Debug — see what keys exist
    impred = list(outputs.values())[0].numpy()  # grab the first tensor

    # Get predicted class
    classcount = np.argmax(impred[0]) + 1  # +1 because dict starts at 1
    
    classdict = {
        1: "Mild Dementia",
        2: "Moderate Dementia",
        3: "No Dementia, Patient is Safe",
        4: "Very Mild Dementia"
    }
    print("Prediction:", classdict[classcount], filepath)

    return render_template(
        'index.html',
        type=classdict[classcount],
        image_file="uploads/" + os.path.basename(filepath)
    )

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
